In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [2]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  regex = re.compile('[^A-Za-zА-Яа-я]')
  cleantext = regex.sub(' ', cleantext)
  return cleantext

In [3]:
def isvotes(tag):
    return tag.has_attr('class') and tag.has_attr('onclick') and not tag.has_attr('href') and not tag.has_attr('type')

In [4]:
def document(id):
    page = requests.get('https://habr.com/ru/post/' + str(id) + '/')
    soup = BeautifulSoup(page.text, 'html5lib')
    info = []
    if soup.find("span", class_= "post__title-text"):
        #title
        info.append(soup.find("span", class_= "post__title-text").text)
        #text
        text = soup.find("div", class_="post__text post__text-html").text
        info.append(cleanhtml(text))
        #year
        info.append(int(soup.find("span", class_= "post__time").text.split()[2]))
        #tags
        tagsshit = soup.find_all("a", class_="inline-list__item-link hub-link")
        tags = ''
        for i in range(len(tagsshit)):
            tags += tagsshit[i].text
        info.append(tags)
        #votes
        a = soup.find(isvotes).text
        if a[0] == '–':
            b = int(a[1:])
            info.append(-b)
        else:
            info.append(int(a))
        #views
        a = soup.find("div", class_="post-stats__views").text.strip()
        if 'k' in a:
            views = float(a[:-1].replace(',', '.')) * 1000
        else:
            views = int(a)
        info.append(views)
        #bookmarks
        info.append(int(soup.find("span", class_="bookmark__counter js-favs_count").text))
        #comments
        comment1 = soup.find_all("div", class_="comment__message")
        comments = ''
        for i in range(len(comment1)):
            comments += comment1[i].text
        info.append(comments)
    else:
        info.append('NaN')
    print(id)
    return info

In [ ]:
from multiprocessing import Pool
with Pool(100) as p:
    a = p.map(document, np.arange(206668, 489000))

In [ ]:
df = pd.DataFrame([a], columns=['title', 'text', 'year', 'tags', 'votes', 'views', 'bookmarks', 'comments'])
df

In [ ]:
df.to_csv('habr.csv')
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
files.download('habr.csv')`